# Use SX5E and bloomberg download data to get me started

In [4]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
#initial cleaning function
def clean_sheet(dataframe):
    adj=dataframe.dropna(axis=1,how='all')
    adj=dataframe.set_index(adj.columns[0]) #use stock tickers as the index
    return adj

In [6]:
#pull the data from excel, form a dictionary of tab names and dataframes
tabs_to_grab=['Quarter','Date','Reported EPS','Comp EPS','Consensus EPS','50d MA','3d return','3d rel return','Volume','Momentum',
             'Revision','Market cap','PE','Property','Price']

data_dict={}

for s in tabs_to_grab:
    data_dict[s]=clean_sheet(pd.read_excel(r'C:\Users\Eric.Li\Documents\Post result data\2016-2018.xlsx',sheet_name=s))

In [7]:
# grab the price time series and transform it into the same datetime format; also generate abs_return, rel_return dataframe
price=data_dict["Price"].dropna(axis=1)
#new_column=[i.strftime("%d/%b/%Y") for i in price.columns.tolist() if type(i)!=str]
#price.columns=new_column


## Signal 2 - based on last quarter's standing

In [12]:
## grab the date 
date=data_dict["Date"].dropna(axis=1,how='all')
adj_date=date.fillna(pd.Timestamp('1900-1-1'))

In [197]:
#extract the bottom and quartile return for all the quarters
start_date=pd.Timestamp('2016-1-1')
end_date=pd.Timestamp('2018-3-31')
per='Q'

dt_rng=pd.date_range(start_date,end_date,freq=per)

def 
rank_dict={}
for s in range(len(dt_rng)-1):
    start_date=dt_rng[s]
    end_date=dt_rng[s+1]
    rank=data_dict['3d rel return'][(adj_date>=start_date) & (adj_date<=end_date)]
    rank_list=[]
    for index,row in rank.iterrows():
        adj_row=row.dropna()
        for s in adj_row.values:
            rank_list.append(s)
    rank_series=pd.Series(rank_list)
    top_quartile=np.percentile(rank_series,80)
    bottom_quartile=np.percentile(rank_series,20)
    rank_dict[end_date]=(top_quartile,bottom_quartile)

rank_dict

{Timestamp('2016-06-30 00:00:00', freq='Q-DEC'): (1.2007525496367955,
  -2.4203371044117548),
 Timestamp('2016-09-30 00:00:00', freq='Q-DEC'): (3.412740954679211,
  -2.436582254931852),
 Timestamp('2016-12-31 00:00:00', freq='Q-DEC'): (2.8935261317512966,
  -2.1637170892269886),
 Timestamp('2017-03-31 00:00:00', freq='Q-DEC'): (2.4805847432361583,
  -2.501572481680293),
 Timestamp('2017-06-30 00:00:00', freq='Q-DEC'): (2.106740738675145,
  -1.2373470254652839),
 Timestamp('2017-09-30 00:00:00', freq='Q-DEC'): (1.5922466374246005,
  -2.492400983840895),
 Timestamp('2017-12-31 00:00:00', freq='Q-DEC'): (2.156497226561612,
  -2.098548353677211),
 Timestamp('2018-03-31 00:00:00', freq='Q-DEC'): (3.381007211752018,
  -1.0606956490463935)}

In [10]:
price.columns[0]

datetime.datetime(2015, 1, 1, 0, 0)

In [196]:
# form the signals by slicing the database into each quarter and merge them together 
for s in range(len(dt_rng)-1):
    print(dt_rng[s+1])
    

2016-03-31 00:00:00
2016-06-30 00:00:00
2016-09-30 00:00:00
2016-12-31 00:00:00
2017-03-31 00:00:00
2017-06-30 00:00:00
2017-09-30 00:00:00
2017-12-31 00:00:00
2018-03-31 00:00:00
